# Heaps

### Overview
|         instance        | persistence | amortization | empty | isEmpty |       insert      |       merge       |           findMin          |     deleteMin     |
|:-----------------------:|:-----------:|:------------:|:-----:|:-------:|:-----------------:|:-----------------:|:--------------------------:|:-----------------:|
|       Leftist Heap      |  ephemeral  |      no      |  O(1) |   O(1)  |     O(log(n))     |     O(log(n))     |            O(1)            |     O(log(n))     |
|      Binomial Heap      |  persistent |      yes     |  O(1) |   O(1)  | O(log(n)) / O(1)* |     O(log(n))     |     O(log(n)) / O(1)**     |     O(log(n))     |
| Scheduled Binomial Heap |  persistent |      no      |  O(1) |   O(1)  |        O(1)       |     O(log(n))     |     O(log(n)) / O(1)**     |     O(log(n))     |
|        Splay Heap       |  ephemeral  |      yes     |  O(1) |   O(1)  | O(n) / O(log(n))* | O(n) / O(log(n))* | O(n) / O(log(n))* / O(1)** | O(n) / O(log(n))* |
|       Pairing Heap      |  ephemeral  |      yes     |  O(1) |   O(1)  |        O(1)       |        O(1)       |            O(1)            | O(n) / O(log(n))* |
|    Lazy Pairing Heap    |  persistent |      yes     |  O(1) |   O(1)  |        TODO       |        TODO       |            O(1)            |        TODO       |

** amortized time*

*** with explicit reference to the minimum element*

### ExplicitMin Heap
`ExplicitMinHeap` is a `Heap` wrapper that explicitly tracks the minimum element of the underlying heap to make the `findMin` access $O(1)$.

Updating the reference to the minimum element takes $O(1)$ time and thus `insert`, `merge` and `deleteMin` have their complexities unchanged (resp. the time complexity of these for the `ExplicitMinHeap` is the same as of the underlying `Heap`).

Tracking the min. element in `deleteMin` involves call to the underlying `findMin` (to lookup new minimum). In case the original `findMin` was slower than `deleteMin`, the performance might degrade. If this is a problem, don't use `ExplicitMinHeap` wrapper and rather update the underlying instance itself.

In [ ]:
class Heap h where

    -- | Construct new (empty) heap
    empty :: Ord a => h a
    
    -- | Check whether a heap is empty
    isEmpty :: h a -> Bool
    
    -- | Add new item to a heap
    insert :: Ord a => a -> h a -> h a
    
    -- | Merge two heaps together
    merge :: Ord a => h a -> h a -> h a
    
    -- | Retrieve the minimum element of a heap
    findMin :: Ord a => h a -> a
    
    -- | Remove the minimum element of a heap and return resulting heap
    deleteMin :: Ord a => h a -> h a


data ExplicitMinHeap h a = Empty | ExplicitMinHeap a (h a)

instance Heap h => Heap (ExplicitMinHeap h) where

    -- | Construct an empty heap in O(1).
    empty = Empty
    
    -- | Check whether given heap is empty.
    isEmpty Empty = True
    isEmpty (ExplicitMinHeap _ h) = isEmpty h && error "Non-empty explicit-min heap with an empty underlying heap"
    
    -- | Add new item to the heap and update the minimum.
    -- |
    -- | Note: Tracking min. element adds only constant overhead and thus does not affect
    -- |       the overall complexity of 'insert'.
    insert x Empty = ExplicitMinHeap x (insert x empty)
    insert x (ExplicitMinHeap y h) = ExplicitMinHeap (min x y) (insert x h)
    
    -- | Merge two heaps together and update the minimum.
    -- |
    -- | Note: Tracking min. element adds only constant overhead and thus does not affect
    -- |       the overall complexity of 'merge'.
    merge (ExplicitMinHeap x h) Empty = ExplicitMinHeap x (merge h empty)
    merge Empty (ExplicitMinHeap x h) = ExplicitMinHeap x (merge empty h)
    merge (ExplicitMinHeap x h1) (ExplicitMinHeap y h2) = ExplicitMinHeap (min x y) (merge h1 h2)
    
    -- | Retrieve the minimum element of a heap in O(1) time.
    -- |
    -- | Note: This improves the complexity of 'findMin' for the base heap to constant.
    findMin Empty = error "Heap is empty"
    findMin (ExplicitMinHeap x _) = x
    
    -- | Remove the minimum element of the underlying heap and lookup new minimum.
    -- |
    -- | Note: Although we do a lookup of new min. element, this does not increase
    -- |       the overall complexity of 'deleteMin' of the underlying heap unless
    -- |       this operation was faster than original 'findMin'.
    deleteMin Empty = error "Heap is empty"
    deleteMin (ExplicitMinHeap _ h) = let h' = deleteMin h in ExplicitMinHeap (findMin h') h'

## Leftist Heap
This implementation is based on a [Leftist tree](https://en.wikipedia.org/wiki/Leftist_tree) without any fancy optimization techniques. Notably, in this implementation `merge` takes two passes:
 1. top-down pass consisting of calls to `merge`
 1. bottom-up pass consisting of calls to `makeNode`

In [ ]:
data LeftistHeap a = Empty | Node Int a (LeftistHeap a) (LeftistHeap a)

-- | Extract the rank of a heap node (zero for empty heap)
rank :: LeftistHeap a -> Int
rank Empty = 0
rank (Node r _ _ _) = r

-- | Create new heap node with given sub-trees while maintaining that
-- | the rank of the left sub-tree is at least as large as the right one.
makeNode :: Ord a => a -> LeftistHeap a -> LeftistHeap a -> LeftistHeap a
makeNode x a b = if ra >= rb then Node (rb + 1) x a b else Node (ra + 1) x b a
    where
        ra = rank a
        rb = rank b

instance Heap LeftistHeap where

    -- | Construct an empty heap in O(1).
    empty = Empty
    
    -- | Check whether given heap is empty in O(1).
    isEmpty Empty = True
    isEmpty _ = False
    
    -- | Add new item to the heap.
    -- |
    -- | The item is first turned to a trivial heap and merged into.
    -- | Therefore this call runs in O(log(n)) steps in the worst case.
    insert x h = merge (Node 1 x Empty Empty) h
    
    -- | Merge two heaps together in O(log(n)) steps.
    -- | Note: The tree is kept balanced because 'makeNode' balances sub-trees via 'rank'.
    merge h Empty = h
    merge Empty h = h
    merge h1 @ (Node _ x a1 b1) h2 @ (Node _ y a2 b2) = 
        if x <= y then makeNode x a1 (merge b1 h2)
        else makeNode y a2 (merge h1 b2)
    
    -- | Retrieve the minimum element of a heap in O(1) worst case time.
    findMin Empty = error "Heap is empty"
    findMin (Node _ x _ _) = x
    
    -- | Remove the minimum element of a heap.
    -- | 
    -- | Becasue the minimum element is the root, this funciton just calls 'merge' on
    -- | both root sub-trees and thus runs in O(log(n)) worst case time.
    deleteMin Empty = error "Heap is empty"
    deleteMin (Node _ x a b) = merge a b

## Binomial Heap
This implementation is based on a set (forest) of binomial trees described for instance [here](https://en.wikipedia.org/wiki/Binomial_heap).

There are no fancy optimization techniques, for instance the heap described below does not track the minimum element.

### Benefits of Binomial Heap
 - Compared to standard *Binary Heap* supports merging (melding) in O(log(n)) time
 - Compared to *Leaftist Heap* has O(1) amortized inserts and merging is based on the size of the larger heap

### Representation
A `BinomialHeap` is a collection of heap-ordered `BinomialTree`s in ascending order of their ranks in which no two trees have the same rank.

A binomial tree is defined recursively as
 - A binomial tree of rank $0$ is a singleton node
 - A binomial tree of rank $r + 1$ if formed by *linking* two binomial trees of rank $r$, making one tree the leftmost child of the other

Alternatively a binomial tree of rank $r$ is a node with $r$ children $t_1, \dots, t_r$ where each $t_i$ is a binomial tree of rank $r - i$ (i.e. the list of children is stored in decreasing order of rank in each `Node`).

Note that we explicitly store rank in each `Node` even though it is not strictly necessary and different representations are also possible. See for instance the one used by the *Scheduled Binomial Heap* which exploits the fact that a binomial tree of rank $r$ contains $2^r$ elements and thus trees in a heap of size $n$ correspond exactly to the 1s in the binary representation of $n$.

From the binary representation follows the observation that there can only be $O(log(n))$ binomial trees in a heap of size $n$ as there can only be that many 1s in the binary representation.

In [ ]:
data BinomialTree a = Node Int a [BinomialTree a]

newtype BinomialHeap a = BinomialHeap [BinomialTree a]

-- | Extract the rank of a binomial tree
rank :: BinomialTree a -> Int
rank (Node r _ _) = r

-- | Extract the root element of a binomial tree
root :: BinomialTree a -> a
root (Node _ x _) = x

-- | Link two binmoial trees of the same rank together in O(1) time.
--
-- | Tree having the greater root is added as the leftmost child of of the other.
-- | Resulting tree has rank incremented by one.
link :: Ord a => BinomialTree a -> BinomialTree a -> BinomialTree a
link t1 @ (Node r x1 c1) t2 @ (Node _ x2 c2) =
    if x1 <= x2 then Node (r + 1) x1 (t2 : c1)
    else Node (r + 1) x2 (t1 : c2)

-- | Insert a tree in given heap.
-- |
-- | Since there are at most O(log(n)) trees, 'link' takes O(1) and in the worst case we traverse
-- | all the trees, the time complexity is logarithmic.
insTree :: Ord a => BinomialTree a -> BinomialHeap a -> BinomialHeap a
insTree t (BinomialHeap []) = BinomialHeap [t]
insTree t (BinomialHeap ts @ (t' : ts')) =
    -- if the tree rank is not present in the heap then just add the tree
    if rank t < rank t' then BinomialHeap (t:ts)
    -- otherwise fold over the heap trees while linking them
    else insTree (link t t') (BinomialHeap ts')

-- | Finds and removes a tree with the minimum element from a heap.
-- |
-- | This function returns both the minimum tree and resulting heap and runs in logarithmic worst case time
-- | as there are only O(log(n)) trees in any heap and we need to check only roots.
removeMinTree :: Ord a => BinomialHeap a -> (BinomialTree a, BinomialHeap a)
removeMinTree (BinomialHeap []) = error "Heap is empty"
removeMinTree (BinomialHeap [t]) = (t, empty)
removeMinTree (BinomialHeap (t:ts)) = let (t', (BinomialHeap ts')) = removeMinTree (BinomialHeap ts) in
    if root t <= root t' then (t, BinomialHeap ts)
    else (t', BinomialHeap (t:ts'))

instance Heap BinomialHeap where
    
    -- | Construct an empty heap in O(1).
    empty = BinomialHeap []
    
    -- | Check whether given heap is empty in O(1).
    isEmpty (BinomialHeap ts) = null ts
    
    -- | Add new item to the heap.
    -- |
    -- | The item is first turned to a trivial binomial tree of rank 0 and inserted into
    -- | the heap via 'insTree'.
    -- |
    -- | The worst case time complexity is O(log(n)) but can be amortized by other updates to just O(1).
    insert x h = insTree (Node 0 x []) h
    
    -- | Merge two heaps together in O(log(n)) steps.
    merge h (BinomialHeap []) = h
    merge (BinomialHeap []) h = h
    merge h1 @ (BinomialHeap (t1:ts1')) h2 @ (BinomialHeap (t2:ts2')) =
        case compare (rank t1) (rank t2) of
            LT -> let (BinomialHeap ts) = merge (BinomialHeap ts1') h2 in BinomialHeap (t1:ts)
            GT -> let (BinomialHeap ts) = merge h1 (BinomialHeap ts2') in BinomialHeap (t2:ts)
            EQ -> insTree (link t1 t2) (merge (BinomialHeap ts1') (BinomialHeap ts2'))
    
    -- | Retrieve the minimum element of a heap in O(log(n)) worst case time.
    -- | Note: The time complexity follows from the analysis of 'removeMinTree'.
    findMin h = let (t, _) = removeMinTree h in root t
    
    -- | Remove the minimum element of a heap in O(log(n)) worst case time.
    -- | Note: The time complexity follows from 'removeMinTree' and 'merge' and the fact that
    -- |       there is at most logarithmic number of trees in a heap (for 'reverse').
    deleteMin h = let ((Node _ _ ts1), h') = removeMinTree h in merge (BinomialHeap (reverse ts1)) h'

## Scheduled Binomial Heap
`ScheduledBinomialHeap` is a persistent variant of the *Binomial Heap* that uses *lazy rebuilding* in combination with *scheduling* to make `insert` run in $O(1)$ worst case time. Other operations on the heap retain their worst case bounds.

### Insertion

Compared to the `BinomialHeap` implementation, here `insTree` is fully incremental (rather than monolithic). Each job in a schedule then represents all the unevaluated suspentions from single call to `insert`.

Because of how `insert` manages the schedule (via `exec`), all required suspensions are memoized prior to calls to `insTree` which makes the whole insertion run in worst case constant time.

### Binomial Tree Representation
In order to make `insTree` incremental, we make an explicit connection between the binomial heaps and binary numbers:
> Trees in a heap correspond to the 1s in the binary representation of the size of the heap.

This way we can
 - create an explicit binary represenation of binomial trees in a heap via `[Digit a]` and eliminate the rank field of the `Node` constructor since a tree in the $i$-th digit has rank $i$ and the children of a rank $r$ node have ranks $r - 1, \dots, 0$
 - make intermediate steps of `insTree` return a suspension with the `Zero` constructor and the last completion step with `One tree`

In [ ]:
data BinomialTree a = Node a [BinomialTree a]

-- | Binary representation of a binomial tree in a heap.
-- |
-- | Additionally, 'Zero's represent intermediate steps of 'insTree' while 'One t' represents a
-- | completed insertion of a 't :: BinomialTree a'.
data Digit a = Zero | One (BinomialTree a)

-- | Suspension schedule in which each job '[Digit a]' represents one partial execution of 'insTree'.
type Schedule a = [[Digit a]]

-- | Final heap is a product of a list of binomial trees in binary representation and a schedule
data ScheduledBinomialHeap a = SBH [Digit a] (Schedule a)

-- | Link two binmoial trees of the same rank together in O(1) time.
--
-- | Tree having the greater root is added as the leftmost child of of the other.
link :: Ord a => BinomialTree a -> BinomialTree a -> BinomialTree a
link t1 @ (Node x1 c1) t2 @ (Node x2 c2) = if x1 <= x2 then Node x1 (t2 : c1) else Node x2 (t1 : c2)

-- | Insert a tree in given heap in constant time.
-- |
-- | Contrary to the implementation used by 'BinomialHeap' this funciton is incremental rather than
-- | monlithic (i.e. each case returns a suspension).
-- |
-- | Moreover, due to the interation between this function and 'exec' in 'insert' all scheduled
-- | suspensions that a tree being matched depens on have been evaluated (and thus memoized)
-- | prior to any call to 'insTree', this function runs in O(1) worst case time.
insTree :: Ord a => BinomialTree a -> [Digit a] -> [Digit a]
insTree t [] = (One t) : []
insTree t (Zero : ds) = (One t) : ds
insTree t ((One t') : ds) = Zero : (insTree (link t t') ds)

-- | Merge two heaps represented by a collection of binomial trees (resp. digits) together.
mrg :: Ord a => [Digit a] -> [Digit a] -> [Digit a]
mrg ds1 [] = ds1
mrg [] ds2 = ds2
mrg (Zero:ds1) (d:ds2) = d : (mrg ds1 ds2)
mrg (d:ds1) (Zero:ds2) = d : (mrg ds1 ds2)
mrg ((One t1):ds1) ((One t2):ds2) = Zero : (insTree (link t1 t2) (mrg ds1 ds2))

-- | Normalize a heap by executing all suspensions in it.
-- |
-- | Note: Suspensions are executed by matching against them. Becaues there can be at most
-- |       O(log(n)) pending suspensions in any heap, this function runs in logarithic time.
normalize :: [Digit a] -> [Digit a]
normalize [] = []
normalize ds @ (_:ds') = seq (normalize ds') ds

-- | Execute a job (suspension) from given schedule in constant time.
-- |
-- | There are two interesting cases:
-- |  1. Since any completed insertion ends with a 'One', when we encounter a 'Zero' we keep
-- |     the rest of an unfinished job in the schedule
-- |  2. Otherwise the insertion if done and we remove the job from the schedule
-- |
-- | Note: It can be proven that by the composition of 'insTree' and 'exec' in 'insert' and
-- |       because 'insTree' creates suspensions, this function runs in O(1) worst case time.
exec :: Schedule a -> Schedule a
exec [] = []
exec ((Zero:job) : schedule) = job : schedule
exec (_:schedule) = schedule

-- | Remove the minimum element from given heap and return it alognside the resulting heap.
-- |
-- | This implemntation is just an adaptation to the binary represenation and runs in O(log(n))
-- | worst case time.
-- |
-- | Note: This function may produce streams with trailing 'Zero's. However, these are either
-- |       discarded by 'findMin' or merged with 'One's by 'deleteMin'.
removeMinTree :: Ord a => [Digit a] -> (BinomialTree a, [Digit a])
removeMinTree [] = error "Tree is empty"
removeMinTree [One t] = (t, [])
removeMinTree (Zero:ds) = let (t', ds') = removeMinTree ds in (t', Zero:ds')
removeMinTree ((One t @ (Node x _)):ds) = case removeMinTree ds of
    (t' @ (Node x' _), ds') -> if x <= x' then (t, Zero:ds) else (t', (One t):ds')

instance Heap ScheduledBinomialHeap where
    
    -- | Construct an empty heap in O(1).
    empty = SBH [] []
    
    -- | Check whether given heap is empty in O(1).
    isEmpty (SBH [] _) = True
    isEmpty _ = False
    
    -- | Add new item to the heap in O(1) worst case time.
    -- |
    -- | Note: After inserting new node to the heap we execute two jobs (suspensions) in the scheudle.
    -- |       The amortized cost of 'insTree' used to be 2 so two executions of scheduled suspensions
    -- |       make all suspension dependencies memoized and in consequence 'insTree' can run in O(1).
    insert x (SBH ds schedule) = let ds' = insTree (Node x []) ds in
        SBH ds' $ exec $ exec (ds' : schedule)
    
    -- | Merge two heaps together in O(log(n)) steps in the worst case.
    -- |
    -- | Note: Resulting heap (after merging) is then trivially normalized (i.e. without optimizations)
    -- |       by executing all suspensions in it and then the schedule is reset. Since 'normalize' runs
    -- |       in at most O(log(n)) steps, this does not add any time complexity.
    merge (SBH ds1 _) (SBH ds2 _) = let ds = normalize (mrg ds1 ds2) in SBH ds []
    
    -- | Retrieve the minimum element of a heap in O(log(n)) worst case time.
    -- |
    -- | Note: The time complexity follows from the analysis of 'removeMinTree' and can be imroved
    -- |       to O(1) with explicit tracking via 'ExplicitMinHeap'.
    findMin (SBH ds _) = let ((Node x _), _) = removeMinTree ds in x
    
    -- | Remove the minimum element of a heap in O(log(n)) worst case time.
    -- | 
    -- | The time complexity follows from 'removeMinTree' and 'mrg' and the fact that
    -- | there is at most logarithmic number of trees in a heap (for 'reverse').
    -- |
    -- | Note: Resulting heap (after merging) is then trivially normalized (i.e. without optimizations)
    -- |       by executing all suspensions in it and then the schedule is reset. Since 'normalize' runs
    -- |       in at most O(log(n)) steps, this does not add any time complexity.
    deleteMin (SBH ds _) = SBH (normalize ds'') []
        where
            ((Node x c), ds') = removeMinTree ds
            ds'' = mrg (map One $ reverse c) ds'

## Splay Heap
This `Heap` instance is based on a [Splay Tree](https://en.wikipedia.org/wiki/Splay_tree).

### Benefits of Splay Heaps
 - Self-balancing (splaying) moves frequently accessed items closer to the root where they can be accessed more quickly in the future
 - One of the fastest heaps when persistence and `merge` are not required (with explicit minimum pointer)
 - Sorting an already sorted array with a `SplayHeap` takes just `O(n)` time
 - Balancing does not require any additional data stored in the tree (resp. nodes)

In [ ]:
data SplayHeap a = Empty | Node (SplayHeap a) a (SplayHeap a)

-- | Splay a pivot element to the root of given Splay tree (heap) and
-- | return a pair of its left sub-tree (smaller than pivot) and the rest (greater than pivot).
-- |
-- | Splaying involves two optimistic balancing operations: Zig-Zig and Zig-Zag and
-- | balances the tree to shorten the leftmost spine (the leftmost element is the minimum).
-- |
-- | Despite splaying, the height of a Splay tree may in the worst case still be linear.
-- | However, the amortized complexity of heap access and update operations is linearithmic.
partition :: Ord a => a -> SplayHeap a -> (SplayHeap a, SplayHeap a)
partition _ Empty = (Empty, Empty)
partition pivot t @ (Node a x b) =
    if x <= pivot then case b of
        Empty -> (t, Empty)
        Node b1 y b2 -> if y <= pivot then let (small, big) = partition pivot b2 in (Node (Node a x b1) y small, big)
                        else let (small, big) = partition pivot b1 in (Node a x small, Node big y b2)
    else case a of
        Empty -> (Empty, t)
        Node a1 y a2 -> if y <= pivot then let (small, big) = partition pivot a2 in (Node a1 y small, Node big x b)
                        else let (small, big) = partition pivot a1 in (small, Node big y (Node a2 x b))

instance Heap SplayHeap where
    
    -- | Construct an empty heap in O(1).
    empty = Empty
    
    -- | Check whether given heap is empty in O(1).
    isEmpty Empty = True
    isEmpty _ = False
    
    -- | Add new item to the heap.
    -- |
    -- | The item is used as a pivot to partition given heap and splayed as new root with
    -- | the smaller elemnts and greater elements as the left and right sub-trees respectively.
    -- |
    -- | As discussed in the 'partition' function, the heap might end up imbalanced, so the
    -- | worst case complexity is O(n) with amortized time complexity O(log(n)).
    insert x h = let (a, b) = partition x h in Node a x b
    
    -- | Merge two heaps together in O(log(n)) amortized steps (O(n) worst case).
    merge Empty h = h
    merge (Node a x b) h = let (ha, hb) = partition x h in Node (merge ha a) x (merge hb b)
    
    -- | Retrieve the minimum element of a heap in O(log(n)) amortized time.
    -- |
    -- | The minimum element in a Splay Tree is the leftmost element so 'findMin' simply
    -- | traverses the leftmost spine in O(log(n)) amortized time.
    -- |
    -- | Note: The time complexity can be reduced to O(1) by wrapping the whole data structure
    -- |       and explicitly tracking a reference to the minimum element.
    findMin Empty = error "Heap is empty"
    findMin (Node Empty x _) = x
    findMin (Node a _ _) = findMin a
    
    -- | Remove the minimum element of a heap.
    -- | 
    -- | The removal procedure is analogous to 'findMin' and with the same argument runs in
    -- | O(log(n)) amortized time.
    deleteMin Empty = error "Heap is empty"
    deleteMin (Node Empty x b) = b
    deleteMin (Node (Node Empty x b) y c) = Node b y c
    deleteMin (Node (Node a x b) y c) = Node (deleteMin a) x (Node b y c)

## Pairing Heap
Implementation of a [Pairing Heap](https://en.wikipedia.org/wiki/Pairing_heap) that does not rely on persistence (i.e. is ephemeral) and does not support `decreaseKey` operation.

### Benefits of Pairing Heaps
 - Similar performance to Splay heaps but much faster when `merge` is used

In [ ]:
data PairingHeap a = Empty | Node a [PairingHeap a]

-- | Merge a list of heaps into single heap in O(n) worst case and O(log(n)) amortized time.
-- |
-- | Merging is done in two passes:
-- |  1. first merge pairs of heaps in the list (hence the name of the heap)
-- |  2. then reverse the direction and merge heaps by folding from the right
mergePairs :: Ord a => [PairingHeap a] -> PairingHeap a
mergePairs [] = Empty
mergePairs [h] = h
mergePairs (h1:h2:hs) = merge (merge h1 h2) (mergePairs hs)

instance Heap PairingHeap where

    -- | Construct an empty heap in O(1).
    empty = Empty
    
    -- | Check whether given heap is empty in O(1).
    isEmpty Empty = True
    isEmpty _ = False
    
    -- | Add new item to the heap in O(1) worst case time.
    -- |
    -- | Added item is first converted to a trivial heap and then merged into the other heap.
    -- | Therefore the time complexity follows from the analysis of 'merge'.
    insert x h = merge (Node x []) h
    
    -- | Merge two heaps together in O(1) steps.
    -- |
    -- | Merging adds the heap with the greater root as the leftmost sub-tree of the other so that
    -- | the minimum of the two is the root of the result.
    merge h Empty = h
    merge Empty h = h
    merge h1 @ (Node x hs1) h2 @ (Node y hs2) =
        if x <= y then Node x (h2:hs1)
        else Node y (h1:hs2)
    
    -- | Retrieve the minimum element of a heap in O(1) time.
    findMin Empty = error "Heap is empty"
    findMin (Node x _) = x
    
    -- | Remove the minimum element of a heap.
    -- |
    -- | The resulting heap is constructed by ignoring current root (the minimum element) and
    -- | merging sub-trees via 'mergePairs', therefore runs in O(log(n)) amortized time (O(n) worst case).
    deleteMin Empty = error "Heap is empty"
    deleteMin (Node _ hs) = mergePairs hs

## Lazy Pairing Heap
`LazyPairingHeap` is a persistent variant of the `PairingHeap`. The formal analysis of this type of heap is rather involved but it's conjectured to be equivalent to the `PairingHeap` in terms of amortized complexity.

This version is slower for languages with strict evaluation semantics but slowdown is less significant in Haskell where lazy evaluation and memization is default for all data types. Also, the cost of laziness is low for applications that heavily utilize persistence.

In [ ]:
data LazyPairingHeap a = Empty | Node a (LazyPairingHeap a) (LazyPairingHeap a)

-- | Add new child to a node.
-- |
-- | If the odd field is empty then the child's put there. Otherwise, the child
-- | is paired with the child in the odd field.
-- |
-- | This is similar to 'mergePairs' but we are parially forcing suspensions on 'merge',
-- | effectively breaking the monolithic 'deleteMin' into lazy and incremental linking.
link :: Ord a => LazyPairingHeap a -> LazyPairingHeap a -> LazyPairingHeap a
link (Node x Empty m) a = Node x a m
link (Node x b m) a = Node x Empty (merge (merge a b) m)

instance Heap LazyPairingHeap where

    -- | Construct an empty heap in O(1).
    empty = Empty
    
    -- | Check whether given heap is empty in O(1).
    isEmpty Empty = True
    isEmpty _ = False
    
    -- | Add new item to the heap in O(1) steps.
    -- |
    -- | Added item is first converted to a trivial heap and then merged into the other heap.
    insert x h = merge (Node x Empty Empty) h
    
    -- | Merge two heaps together.
    -- |
    -- | This is done using 'link' which adds the greater root as a child to the other heap.
    merge a Empty = a
    merge Empty b = b
    merge a @ (Node x _ _) b @ (Node y _ _) = if x <= y then link a b else link b a
    
    -- | Retrieve the minimum element of a heap in O(1) time.
    findMin Empty = error "Heap is empty"
    findMin (Node x _ _) = x
    
    -- | Remove the minimum element of a heap.
    deleteMin Empty = error "Heap is empty"
    deleteMin (Node _ a m) = merge a m